# Westminster dog show data
## Part 2: Group winners
Grabbing the data from the group winners from 1924 - 2023
https://www.westminsterkennelclub.org/about-sensation/history/herding-group-winners

In [9]:
# make ur life easier libs
from tqdm import tqdm
from pathlib import Path
import pprint as pp

# analysis
import requests
import lxml.html
import pandas as pd
import re


In [93]:
# url selection nested within script? not able to chain select from index.
index_URL = "https://www.westminsterkennelclub.org/tag?id=5afcae7ddcb2e405b315c336&name=Group%20Winners"

# setting up
base_URL = "https://www.westminsterkennelclub.org/about-sensation/history/"
rel_urls = ["sporting-group-winners",
            "hound-group-winners",
            "working-group-winners",
            "terrier-group-winners",
            "toy-group-winners",
            "non-sporting-group-winners",
            "herding-group-winners",
           ]

# adapted from Jeremy's script - grabbing + storing in HTML files.
## maybe do it concurrently?????? if list of URLs was long. Refer to Kai's doc.

for url in tqdm(rel_urls):
    dest = Path("table-pgs/" + f"{url}" + ".html")
    
    if dest.exists(): # ... load it from file
        page_html = open(dest).read()
        print(f"file \"{url}\" exists! Loading\n")
        print("-------------------------------")
        page_html = open(dest).read()

        ## CALL A FUNCTION? 
            
    else: # ... fetch it
        page_url = base_URL + url
        print("Fetching " + page_url)
        page_html = requests.get(page_url).text
        
        # ... and then save it to file
        with open(dest, "w") as f:
            f.write(page_html)

100%|███████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 1052.33it/s]

file "sporting-group-winners" exists! Loading

-------------------------------
file "hound-group-winners" exists! Loading

-------------------------------
file "working-group-winners" exists! Loading

-------------------------------
file "terrier-group-winners" exists! Loading

-------------------------------
file "toy-group-winners" exists! Loading

-------------------------------
file "non-sporting-group-winners" exists! Loading

-------------------------------
file "herding-group-winners" exists! Loading

-------------------------------


In [92]:
pp.pprint(page_html)

# creating lxml object
group_dom = lxml.html.fromstring(page_html)
print(group_dom.text)

('        <!DOCTYPE html>\n'
 '        <!-- Templates version v.2.9.0 -->\n'
 '        <html lang="en"  >\n'
 '            <head>\n'
 '                <meta charset="utf-8"/>\n'
 '                <meta name="viewport" content="width=device-width, '
 'initial-scale=1.0, maximum-scale=6.0"/>\n'
 '                <meta http-equiv="X-UA-Compatible" content="IE=edge"/>\n'
 '            \n'
 '            \n'
 '            \n'
 '                <meta name="msapplication-config" '
 'content="/static/favicon/browserconfig.xml?q=77fc23c3ad" />\n'
 '                <link rel="apple-touch-icon" sizes="180x180" '
 'href="/static/favicon/apple-touch-icon.png?q=77fc23c3ad">\n'
 '                <link rel="android-touch-icon" sizes="180x180" '
 'href="/static/favicon/android-touch-icon.png?q=77fc23c3ad">\n'
 '                <link rel="icon" type="image/png" '
 'href="/static/favicon/favicon-32x32.png?q=77fc23c3ad" sizes="32x32">\n'
 '                <link rel="icon" type="image/png" '
 'href="/static

In [120]:
# re to match group
re_exp = '.*(?=\-group-winners)'
m = re.search(re_exp, "non-sporting-group-winners")
print(m.group(0))

non-sporting


In [123]:
group_table = group_dom.cssselect("table")[0]
rows = group_table.cssselect('tr')[1:]

# manually removing faulty rows
del rows[3]

group_winners = []
index = 0

for row_el in rows:
    champion = {}
    children = row_el.getchildren()
    year = children[0].text
    breed = children[1].text
    dog = children [2].text
    owners = children[3].text
    
    # # TROUBLE SHOOTING
    # print(f"Currently on row: {index}" )
    # print(year, breed, dog, owners)
    # print("list is " + str(len(children)) + " items long")
    # print("-----------------------------")
    # index = index + 1
    
    champion['breed'] = breed
    champion['dog'] = dog
    champion['year'] = year
    champion['owners'] = owners

    group_winners.append(champion)

pp.pprint(group_winners)

[{'breed': 'Australian Shepherd',
  'dog': "GCHP CH Northbay Xsell That's A Wrap",
  'owners': 'Bette A Evans',
  'year': '2023'},
 {'breed': 'German Shepherd Dog',
  'dog': 'GCHG CH Gem-N-I River of Urloved CGC',
  'owners': 'Richard A Waits',
  'year': '2022'},
 {'breed': 'Old English Sheepdog',
  'dog': "GCH CH Bugaboo's Courage of Conviction",
  'owners': 'Colton Johnson & Heather Johnson & Douglas Johnson & Michaelanne '
            'Johnson',
  'year': '2021'},
 {'breed': 'Shetland Sheepdog',
  'dog': 'GCHP CH. Syringa- Akadia The Corsair',
  'owners': 'Tyler Crady & Deborah Sirdofsky & Guy Okada & Debi Tario',
  'year': '2020'},
 {'breed': 'Bouvier Des Flandres',
  'dog': "GCh. Quiche's Major League",
  'owners': 'Julianna Garrison & Elaine & Louise Paquette',
  'year': '2019'},
 {'breed': 'Border Collie',
  'dog': 'GCh.\xa0 Majestic Elite Clever Endeavor',
  'owners': 'Ricki Johnson & J Frank Baylis & Ronni Delay',
  'year': '2018'},
 {'breed': 'German Shepherd Dog',
  'dog': '